In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
import keras
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
# from keras.callbacks import *

In [ ]:
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.2
set_session(tf.Session(config=config))

## LSTM to detect voluntary blinks


### Dataset

In [ ]:
dataset = np.load(f'dataset/dataset.npy', allow_pickle=True).item()
x_train = dataset['x_train']
y_train = dataset['y_train']
x_test = dataset['x_test']
y_test = dataset['y_test']

### Architecture

In [ ]:
train_data = {}
train_data['name'] = 'train_1_orig'
train_data['model'] = [('lstm',256),('dropout',0.1),('lstm',128),('dense','softmax',5)]

In [ ]:
model = Sequential()

for i, layer in enumerate(train_data['model']):
    if layer[0]=='lstm':
        if i==0:
            model.add(LSTM(units=layer[1], return_sequences=True, input_shape=(x_train.shape[1],1)))
        else:
            model.add(LSTM(units=layer[1]))
    elif layer[0]=='dropout':
        model.add(Dropout(layer[1]))
    elif layer[0]=='dense':             
        model.add(Dense(layer[2], activation=layer[1]))

mc_val = ModelCheckpoint(f'models/{train_data["name"]}.h5', monitor='val_accuracy', mode='max', verbose = 1, save_best_only=True)  #Save best model accuracy

model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
model.build()

In [ ]:
model.summary()

### Training

In [ ]:
class_weight = {0: 4 , 1: 4, 2: 4, 3: 1}
history = model.fit(x_train,y_train,validation_split = 0.1,epochs = 100,batch_size = 64, callbacks=[mc_val])

### Save the model

In [ ]:
model.save(f'models/{train_data["name"]}.h5')

### Curves and metrics

In [ ]:
plt.clf()
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.clf()
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

In [ ]:
window_size = 5
i = 0
loss_p = []
valoss_p = []
acc_p = []
valacc_p = []
while i < len(loss) - window_size + 1:
    l_window = loss[i : i + window_size]
    window_avr  = sum(l_window) / window_size
    loss_p.append(window_avr)
    lv_window = val_loss[i : i + window_size]
    window_avr1  = sum(lv_window) / window_size
    valoss_p.append(window_avr1)

    a_window = acc[i : i + window_size]
    window_avr2  = sum(a_window) / window_size
    acc_p.append(window_avr2)
    av_window = val_acc[i : i + window_size]
    window_avr3  = sum(av_window) / window_size
    valacc_p.append(window_avr3)
    
    i += 1
print(len(loss_p))
print(len(acc_p))
epochs = range(1, len(loss_p) + 1) 
plt.plot(epochs, loss_p, 'g', label='Training loss')
plt.plot(epochs, valoss_p, 'y', label='Validation loss')
plt.title('Training_Validation loss AVR')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(epochs, acc_p, 'g', label='Training acc')
plt.plot(epochs, valacc_p, 'y', label='Validation acc')
plt.title('Training_Validation acc AVR')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
train_data['loss_train'] = np.array(loss_p)
train_data['loss_val'] = np.array(valoss_p)
train_data['acc_train'] = np.array(acc_p)
train_data['acc_val'] = np.array(valacc_p)

In [ ]:
#############Making predictions with our model#############
y_pred = model.predict(x_test)
ypred = np.argmax(y_pred, axis = 1)[:450] 
ytest = np.argmax(y_test,axis = 1)[:450] 

# print(ypred) 
# print(ytest)
# print("The last array: actual Y_values!")

###############ROC AUC binary classif#################
auc_val = roc_auc_score(y_test, y_pred, multi_class="ovr",average="macro")
print('AUC')
print(auc_val)
train_data['auc'] = auc_val


In [ ]:
print(len(ypred))
tp_blnk = 0 #True Positive blink
fn = 0 #FalseNegative
fp = 0 #FalsePositive
lbel = 4
for i in range(len(ypred)):
  if(ytest[i]==lbel and ypred[i]==lbel):
    tp_blnk+=1   
  elif(ytest[i]==lbel and ypred[i]!=lbel):
    fn+=1
  elif(ytest[i]!=lbel and ypred[i]==lbel):
    fp+=1     
print(fn)
print(fp)
print(tp_blnk)
prec = tp_blnk/(tp_blnk+fp)
recal = tp_blnk/(tp_blnk+fn)
print("Precision: ", prec)
print("Recal: ", recal)

train_data['precision'] = prec
train_data['recall'] = recal

In [ ]:
np.save(f'curves/{train_data["name"]}.npy',train_data)